In [1]:
from lxml import etree
import os 
import cv2

In [2]:
xmlsPath = '/Users/sandeep/Downloads/Data5/boxlabel'
imagesPath = '/Users/sandeep/Downloads/Data5/Image'
images = os.listdir(imagesPath)
xmls = os.listdir(xmlsPath)
#filter imagess
images = [i for i in images if f'{i[0:-4]}.xml' in xmls]

In [3]:
def getbboxCoord(obj):
    bbox = obj[1]
    return [int(c.text) for c in bbox.getchildren()] 
def getCoord(result):
    return [getbboxCoord(obj) for obj in result] 
def getlabel(image):
    imageLabelPath = '/Users/sandeep/Downloads/Data5/boxlabel/' + image[0:-4]+'.xml'
    imageXMLFile = etree.parse(imageLabelPath)
    x,y = 0,0
    #Get x
    path = "/annotation/object"
    result = imageXMLFile.xpath(path)
    bboxs = getCoord(result)
    return bboxs

In [4]:
def getImagePath(image):
    return '/Users/sandeep/Downloads/Data5/Image' + '/' + image

def getLabel(image):
    c = image[0:-4].split(',')
    return [int(c[0]),int(c[1])]    
def loadImages():
    return [cv2.imread(getImagePath(i)) for i in images]
def loadLabels():
    return [getlabel(i) for i in images]

#Loading Test Images and Lables
imgs  = loadImages()
labels = loadLabels()
    

In [16]:
def writeResults(fname,content):
    basePath = '/Users/sandeep/Desktop/dataandmodles/data/playerTest/maskrcnn'
    with open(f'{basePath}/{fname}.txt' , 'w') as f:
        f.write(content)

#AP

In [6]:
import numpy as np
from copy import deepcopy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [7]:
def calc_iou(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = abs(max((xB - xA, 0)) * max((yB - yA), 0))
    if interArea == 0:
        return 0
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = abs((boxA[2] - boxA[0]) * (boxA[3] - boxA[1]))
    boxBArea = abs((boxB[2] - boxB[0]) * (boxB[3] - boxB[1]))

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [8]:
def calc_precision_recall(image_results):
    """Calculates precision and recall from the set of images
    Args:
        img_results (dict): dictionary formatted like:
            {
                'img_id1': {'true_pos': int, 'false_pos': int, 'false_neg': int},
                'img_id2': ...
                ...
            }
    Returns:
        tuple: of floats of (precision, recall)
    """
    true_positive=0
    false_positive=0
    false_negative=0
    for img_id, res in image_results.items():
        true_positive +=res['true_positive']
        false_positive += res['false_positive']
        false_negative += res['false_negative']
        try:
            precision = true_positive/(true_positive+ false_positive)
        except ZeroDivisionError:
            precision=0.0
        try:
            recall = true_positive/(true_positive + false_negative)
        except ZeroDivisionError:
            recall=0.0
    return (precision, recall)

In [9]:
def get_single_image_results(gt_boxes, pred_boxes, iou_thr):
    """Calculates number of true_pos, false_pos, false_neg from single batch of boxes.
    Args:
        gt_boxes (list of list of floats): list of locations of ground truth
            objects as [xmin, ymin, xmax, ymax]
        pred_boxes (dict): dict of dicts of 'boxes' (formatted like `gt_boxes`)
            and 'scores'
        iou_thr (float): value of IoU to consider as threshold for a
            true prediction.
    Returns:
        dict: true positives (int), false positives (int), false negatives (int)
    """
    all_pred_indices= range(len(pred_boxes))
    all_gt_indices=range(len(gt_boxes))
    if len(all_pred_indices)==0:
        tp=0
        fp=0
        fn=0
        return {'true_positive':tp, 'false_positive':fp, 'false_negative':fn}
    if len(all_gt_indices)==0:
        tp=0
        fp=0
        fn=0
        return {'true_positive':tp, 'false_positive':fp, 'false_negative':fn}
    
    gt_idx_thr=[]
    pred_idx_thr=[]
    ious=[]
    for ipb, pred_box in enumerate(pred_boxes):
        for igb, gt_box in enumerate(gt_boxes):
            iou= calc_iou(gt_box, pred_box)
            
            if iou >iou_thr:
                gt_idx_thr.append(igb)
                pred_idx_thr.append(ipb)
                ious.append(iou)
    iou_sort = np.argsort(ious)[::1]
    if len(iou_sort)==0:
        tp=0
        fp=0
        fn=0
        return {'true_positive':tp, 'false_positive':fp, 'false_negative':fn}
    else:
        gt_match_idx=[]
        pred_match_idx=[]
        for idx in iou_sort:
            gt_idx=gt_idx_thr[idx]
            pr_idx= pred_idx_thr[idx]
            # If the boxes are unmatched, add them to matches
            if(gt_idx not in gt_match_idx) and (pr_idx not in pred_match_idx):
                gt_match_idx.append(gt_idx)
                pred_match_idx.append(pr_idx)
        tp= len(gt_match_idx)
        fp= len(pred_boxes) - len(pred_match_idx)
        fn = len(gt_boxes) - len(gt_match_idx)
    return {'true_positive': tp, 'false_positive': fp, 'false_negative': fn}

In [10]:
def get_model_scores(pred_boxes):
    """Creates a dictionary of from model_scores to image ids.
    Args:
        pred_boxes (dict): dict of dicts of 'boxes' and 'scores'
    Returns:
        dict: keys are model_scores and values are image ids (usually filenames)
    """
    model_score={}
    for img_id, val in pred_boxes.items():
        for score in val['scores']:
            if score not in model_score.keys():
                model_score[score]=[img_id]
            else:
                model_score[score].append(img_id)
    return model_score

In [11]:
def get_avg_precision_at_iou(gt_boxes, pred_bb, iou_thr=0.5):

    model_scores = get_model_scores(pred_bb)
    sorted_model_scores= sorted(model_scores.keys())
# Sort the predicted boxes in descending order (lowest scoring boxes first):
    for img_id in pred_bb.keys():
        arg_sort = np.argsort(pred_bb[img_id]['scores'])
        pred_bb[img_id]['scores'] = np.array(pred_bb[img_id]['scores'])[arg_sort].tolist()
        pred_bb[img_id]['boxes'] = np.array(pred_bb[img_id]['boxes'])[arg_sort].tolist()
    pred_boxes_pruned = deepcopy(pred_bb)
    
    precisions = []
    recalls = []
    model_thrs = []
    img_results = {}
    
    img_ids = gt_boxes.keys()
    for img_id in img_ids:   
        # Recalculate image results for this image
        img_results[img_id] = get_single_image_results(gt_boxes[img_id], pred_boxes_pruned[img_id]['boxes'], iou_thr=iou_thr)
# calculate precision and recall
    #print(img_results)
    prec, rec = calc_precision_recall(img_results)
    precisions.append(prec)
    recalls.append(rec)
  
    precisions = np.array(precisions)
    recalls = np.array(recalls)
    prec_at_rec = []
    for recall_level in np.linspace(0.0, 1.0, 11):
        try:
            args= np.argwhere(recalls>recall_level).flatten()
            prec= max(precisions[args])
            print(recalls,"Recall")
            print(recall_level,"Recall Level")
            print(args, "Args")
            print(prec, "precision")
        except ValueError:
            prec=0.0
        prec_at_rec.append(prec)
    avg_prec = np.mean(prec_at_rec) 
    return {
        'avg_prec': avg_prec,
        'precisions': precisions,
        'recalls': recalls,
        'model_thrs': model_thrs}

In [14]:
# gt_boxes= {"img_00285.png": [[480, 457, 515, 529], [637, 435, 676, 536]]}
# #Pred Boxes
# pred_boxs={"img_00285.png": {"boxes": [[330, 463, 387, 505], [356, 456, 391, 521], [420, 433, 451, 498], [328, 465, 403, 540], [480, 477, 508, 522], [357, 460, 417, 537], [344, 459, 389, 493], [485, 459, 503, 511], [336, 463, 362, 496], [468, 435, 520, 521], [357, 458, 382, 485], [649, 479, 670, 531], [484, 455, 514, 519], [641, 439, 670, 532]], "scores": [0.0739, 0.0843, 0.091, 0.1008, 0.1012, 0.1058, 0.1243, 0.1266, 0.1342, 0.1618, 0.2452, 0.8505, 0.9113, 0.972]}}

In [15]:
# get_avg_precision_at_iou(gt_boxes,pred_boxs)

[1.] Recall
0.0 Recall Level
[0] Args
0.14285714285714285 precision
[1.] Recall
0.1 Recall Level
[0] Args
0.14285714285714285 precision
[1.] Recall
0.2 Recall Level
[0] Args
0.14285714285714285 precision
[1.] Recall
0.30000000000000004 Recall Level
[0] Args
0.14285714285714285 precision
[1.] Recall
0.4 Recall Level
[0] Args
0.14285714285714285 precision
[1.] Recall
0.5 Recall Level
[0] Args
0.14285714285714285 precision
[1.] Recall
0.6000000000000001 Recall Level
[0] Args
0.14285714285714285 precision
[1.] Recall
0.7000000000000001 Recall Level
[0] Args
0.14285714285714285 precision
[1.] Recall
0.8 Recall Level
[0] Args
0.14285714285714285 precision
[1.] Recall
0.9 Recall Level
[0] Args
0.14285714285714285 precision


{'avg_prec': 0.12987012987012986,
 'precisions': array([0.14285714]),
 'recalls': array([1.]),
 'model_thrs': []}